In [1]:
# | default_exp tools.execute_bash

In [2]:
# | export
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory

import subprocess
from typing import Dict, Any, Optional
from pydantic import BaseModel, ValidationError
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

class ExecuteBashParams(BaseModel):
    command: str
    working_dir: Optional[str] = None
    timeout: Optional[int] = 60
    env_vars: Optional[Dict[str, str]] = None
    capture_output: Optional[bool] = True
    shell: Optional[str] = "/bin/bash"
    summary: Optional[str] = None

class ExecuteBashTool(BaseTool):
    def __init__(self):
        metadata = ToolMetadata(
            name="execute_bash",
            description="Execute bash commands with safety controls",
            category=ToolCategory.SYSTEM,
            requires_approval=False,  # Disabled for automated testing
            is_dangerous=False        # Disabled for automated testing
        )
        super().__init__(metadata)

    def get_parameters_schema(self) -> Dict[str, Any]:
        try:
            return {
                "type": "object",
                "properties": {
                    "command": {"type": "string", "description": "Bash command to execute"},
                    "working_dir": {"type": "string", "description": "Working directory for command execution"},
                    "timeout": {"type": "integer", "description": "Timeout in seconds (default: 30)"},
                    "env_vars": {"type": "object", "description": "Additional environment variables"},
                    "capture_output": {"type": "boolean", "description": "Capture stdout/stderr (default: true)"},
                    "shell": {"type": "string", "description": "Shell to use (default: /bin/bash)"},
                    "summary": {"type": "string", "description": "Brief description of what the command does"}
                },
                "required": ["command"]
            }
        
        except Exception as e:
            logger.error(f"Schema generation failed: {e}")
            return {}

    def execute(self, **kwargs) -> Dict[str, Any]:
        try:
            params = ExecuteBashParams(**kwargs)
            shell = params.shell or "/bin/bash"
            result = subprocess.run(
                params.command,
                shell=True,
                capture_output=params.capture_output,
                text=True,
                timeout=params.timeout,
                cwd=params.working_dir,
                env={**os.environ, **(params.env_vars or {})} if params.env_vars else None
            )
            return {
                "success": True,
                "stdout": result.stdout,
                "stderr": result.stderr,
                "exit_status": result.returncode,
                "summary": params.summary or f"Executed command: {params.command}"
            }
        except subprocess.TimeoutExpired as e:
            logger.warning(f"Command timed out: {params.command}")
            return {"success": False, "error": f"Command timed out after {params.timeout} seconds"}
        except subprocess.CalledProcessError as e:
            logger.error(f"Subprocess error: {str(e)}")
            return {"success": False, "error": f"Command execution failed: {str(e)}", "stdout": e.stdout, "stderr": e.stderr}
        except ValidationError as e:
            logger.error(f"Invalid parameters: {str(e)}")
            return {"success": False, "error": f"Invalid parameters: {str(e)}"}
        except Exception as e:
            logger.error(f"Unexpected execution error: {str(e)}")
            return {"success": False, "error": f"Unexpected error: {str(e)}"}